In [ ]:
import scipy
import scipy.signal as signal
import numpy as np
import matplotlib.pyplot as plt

#finite impulse response, infinite impulse response

In [ ]:
#均衡滤波器参数, IIR滤波器的系数为二次均衡滤波器的系数

a = np.array([1.0, -1.947463016918843, 0.9555873701383931])
b = np.array([0.9833716591860479, -1.947463016918843, 0.9722157109523452])

# 44.1kHz， 1秒的频率扫描波
t = np.arange(0, 0.5, 1/44100.0)
x= signal.chirp(t, f0=10, t1 = 0.5, f1=1000.0)

plt.plot(t, x)
plt.show()

# 直接一次计算滤波器的输出
y = signal.lfilter(b, a, x)

# 将输入信号分为50个数据一组
x2 = x.reshape((-1,50))

# 滤波器的初始状态为0， 长度是滤波器系数长度-1
z = np.zeros(max(len(a),len(b))-1, dtype=np.float)
y2 = [] # 保存输出的列表

for tx in x2:
    # 对每段信号进行滤波，并更新滤波器的状态z
    ty, z = signal.lfilter(b, a, tx, zi=z)
    # 将输出添加到输出列表中
    y2.append(ty)

# 将输出y2转换为一维数组
y2 = np.array(y2)
y2 = y2.reshape((-1,))

# 输出y和y2之间的误差
print(np.sum((y-y2)**2))

# 绘图
plt.plot(t, y, t, y2)
plt.title('wave after filter')
plt.show()

In [ ]:
#如果将一个脉冲信号输入到滤波器中，所得到的输出被称为滤波器的其脉冲响应。
#所谓脉冲信号就是在时刻0为1，其余时刻均为0的信号。
#根据FIR滤波器的公式，FIR滤波器的脉冲响应就是滤波器的系数。
#而IIR滤波器的脉冲响应就不是很直观了，
#下面使用lfilter计算IIR滤波器的脉冲响应，其中的IIR滤波器的系数和前面的一样
impulse = np.zeros(1000, dtype=np.float)
impulse[0] = 1
#impulse 就是 脉冲信号

h = signal.lfilter(b, a, impulse)
plt.plot(h)
#二次IIR均衡器的脉冲响应
plt.show()

#将h当作FIR滤波器的系数对信号x进行滤波
y3 = signal.lfilter(h, 1, x) # FIR 模拟 IIR(二次均衡滤波器系数)
plt.plot(y3)
plt.show()


In [ ]:
#测量足够长的脉冲响应，就可以用FIR滤波器足够精确地模拟IIR滤波器
#IIR滤波器的脉冲响应是随着滤波器长度呈指数衰减的

import scipy.signal as signal
import numpy as np
import pylab as pl

# 某个均衡滤波器的参数
a = np.array([1.0, -1.947463016918843, 0.9555873701383931])
b = np.array([0.9833716591860479, -1.947463016918843, 0.9722157109523452])

ns = range(10, 1100, 100)
err = []

for n in ns:
    # 计算脉冲响应
    impulse = np.zeros(n, dtype=np.float)
    impulse[0] = 1
    h = signal.lfilter(b, a, impulse) #IIR

    # 直接FIR滤波器的输出
    y2 = signal.lfilter(h, 1, x) #FIR

    # 输出y和y2之间的误差
    err.append(np.sum((y-y2)**2))

# 绘图
pl.figure(figsize=(8,4))
pl.semilogy(ns , err, "-o")
pl.xlabel("impulse length")
pl.ylabel("difference between FIR and IIR")
pl.show()

理想低通滤波器的频率响应

<p> 理想低通滤波器的脉冲响应为(？？)：
    $$h_{ideal}(n) = \frac {sin(2 * \pi * f_c * n)} {\pi * n} = 2 * f_c * sinc(2 * f_c * n)$$
    $$sinc(x) = \sin(\pi x)/(\pi x)$$
<p>  其中$f_c$的含义就是每个取样点所包含的信号的周期数
    
<p> 取$h_{ideal}$中$0<=n < L$ 个值作为低通FIR滤波器的系数

In [ ]:
# FIR滤波器设计 ？？
import scipy.signal as signal
import numpy as np
import pylab as pl

num_param = 1024

def h_ideal(n, fc):
    # fc = flow/maxf
    #np.sinc = sin(pi*x)/pi*x
    return 2*fc*np.sinc(2*fc*np.arange(0, n, 1.0))  #参考上面公式

b = h_ideal(num_param, 0.25)

b_fft = np.fft.fft(b)
pl.figure(figsize=(8,3))
pl.ylabel(u"mag(dB)")
pl.plot(np.abs(b_fft)[0:int(num_param/2)])

w, h = signal.freqz(b) #  Compute the frequency response of a digital filter？什么是数字滤波器的频率响应, 对b进行了加窗运算

pl.figure(figsize=(8,3))
print("len(w):", len(w), "len(h):", len(h))
pl.plot(w/np.pi/2, np.abs(h)) # ?? w / 2 * pi， 归一化处理？
pl.xlabel(u"normalized freq T/samples")
pl.ylabel(u"mag(dB)")
pl.show()

pl.plot(w/2/np.pi, 20*np.log10(np.abs(h)))
pl.xlabel(u"normalized freq T/samples")
pl.ylabel(u"mag(dB)")
pl.show()

In [ ]:

#显然此频率响应和理想的低通滤波器相差甚远，并且即使增加FIR滤波器的系数也没有作用。
#因为我们舍弃了n<0的那一半系数，而这些系数有着相当大的影响，
#因此只截取n>=0的部分是不够的，如果我们将n<0的那一半系数也添加进滤波器的话，得到的频率响应将会有很大的改善
def h_ideal(n, fc):
    return 2*fc*np.sinc(2*fc*np.arange(-n, n, 1.0)) # 取 -n 到 n

b = h_ideal(32, 0.25) #脉冲响应

w, h = signal.freqz(b) #  Compute the frequency response of a digital filter

pl.figure(figsize=(8,3))

pl.plot(w/2/np.pi, np.abs(h))
pl.xlabel(u"normalized freq T/samples")
pl.ylabel(u"mag")
pl.show()

pl.plot(w/2/np.pi, 20*np.log10(np.abs(h)))
pl.xlabel(u"normalized freq T/samples")
pl.ylabel(u"mag(dB)")
pl.show()


impulse = np.zeros(64, dtype=np.float)
impulse[0] = 1
#impulse 就是 脉冲信号
y = signal.lfilter(b, 1, impulse) # y 为 参数为b的FIR的脉冲响应， 也就是FIR的参数b 即：y == b
pl.plot(y)
pl.show()

pl.plot(b)
pl.show()

b_fft = scipy.fftpack.fft(b)
pl.plot(np.abs(b_fft)[0:int(len(b_fft)/2)])
pl.show()

In [ ]:
# 用firwin设计滤波器
# 为了频率响应更好必须增加滤波器的点数，然而为了减少延时，必须减少点数，为了解决这个矛盾，我们给系数乘上一个窗函数，让它快速收敛
# firwin(N, cutoff, width=None, window='hamming')
# 其中N为滤波器的长度；cutoff为以正规化的频率；window为所使用的窗函数

import scipy.signal as signal
import numpy as np
import pylab as pl

def h_ideal(n, fc):
    return 2*fc*np.sinc(2*fc*np.arange(-n, n, 1.0))

b = h_ideal(30, 0.25) # 以fs正规化的频率
b2 = signal.firwin(len(b), 0.5) # 以fs/2正规化的频率

w, h = signal.freqz(b)
w2, h2 = signal.freqz(b2)

pl.figure(figsize=(8,6))
pl.subplot(211)
pl.plot(w/2/np.pi, 20*np.log10(np.abs(h)), label=u"h_ideal")
pl.plot(w2/2/np.pi, 20*np.log10(np.abs(h2)), label=u"firwin")
pl.xlabel(u"Normalized Freq T/sample")
pl.ylabel(u"mag(dB)")
pl.legend()
pl.subplot(212)
pl.plot(b, label=u"h_ideal")
pl.plot(b2, label=u"firwin")
pl.legend()
pl.show()


#remez函数能够帮助我们找到更优的滤波器系数
# remez(numtaps, bands, desired,
#    weight=None, Hz=1, type='bandpass', maxiter=25, grid_density=16)

<p> 滤波器级联: 级联一个高通与一个低通，生成一个带通滤波器

* 假设有两个滤波器h1和h2，我们将h1的输出输入到h2，这样得到的滤波器称为h1和h2的级联。
* 级联后的滤波器的脉冲响应为h1和h2的脉冲响应的卷积，而其频率响应为两个滤波器的频率响应的乘积

<p> 数字滤波是将输入信号与数字滤波器脉冲响应进行卷积(FIR)
<p> 脉冲响应以这种方法应用,有个特定的名字： 滤波器核
<p> 递归滤波器, 每个采样点通过将输入权重相加,并且还使用先前计算的输出值。递归滤波器的脉冲响应由正弦波组成，并且幅度指数衰减

In [ ]:
# IIR滤波器设计

In [ ]:
#脉冲
n = 1024
x = np.linspace(0, 1, n)
impulse = np.zeros(1024, dtype=np.float)
impulse[0] = 1
#impulse 就是 脉冲信号
impulse_fft = scipy.fftpack.fft(impulse)

plt.plot(x, np.abs(impulse_fft)) # 全频域
plt.show()

fc = 0.25

def h_ideal(n, fc):
    return 2*fc*np.sinc(2*fc*np.arange(-n, n, 1.0))

x = np.linspace(0, 1, 100)
y = np.zeros(len(x))
y[0:25] = 1
plt.plot(x, y)
plt.show()

y_fft = scipy.fftpack.fft(y)
plt.plot(x, np.abs(y_fft))
plt.show()

#plt.ylim(-1, 1)
plt.show()

In [ ]:
help(signal.freqz)

Reference: https://wizardforcel.gitbooks.io/hyry-studio-scipy/content/18.html

<p> https://www.zhihu.com/question/29461110?sort=created>https://www.zhihu.com/question/29461110?sort=created